## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
# !wget "https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1"

In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1.1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [5]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [6]:
# method most_simmilar
wv_embeddings.most_similar('dog', topn=5)

[('animal', 0.8564179539680481),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.7607938647270203)]

Вывод: слово `cat` не входит в топ-5 близких слов к слову `dog` по языковой модели StackOverFlow.

Посмотрим другие языковые модели и возьмем модель Твитера (2B tweets, 27B tokens, 1.2M vocab)

In [7]:
import gensim.downloader


model_info = gensim.downloader.info()
model_keys = list(model_info['models'].keys())

print("Список всех моделей: ", model_keys)

#Скачаем модель твитерра как в лекции

model_twitter = gensim.downloader.load('glove-twitter-100')

model_twitter.save('glove_twitter_100_model')

Список всех моделей:  ['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [8]:
model_twitter.most_similar('dog', topn=5)

[('cat', 0.8752090334892273),
 ('dogs', 0.8713089823722839),
 ('puppy', 0.8299306631088257),
 ('pet', 0.805988609790802),
 ('cats', 0.7908133268356323)]

Вывод: тут слово `cat` на первом месте.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [9]:
import numpy as np
import re
from nltk.tokenize import WordPunctTokenizer
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        tk = WordPunctTokenizer() 
        return tk.tokenize(text) 
#         return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [10]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    question = tokenizer.tokenize(question)
    vector = np.zeros(dim)
    count = 0 
    
    for word in question:
        if word in embeddings:
            vector += embeddings[word]
            count += 1
    
    if count == 0:
        return np.zeros(dim)
    
    return vector / count


Тест

In [11]:
question_to_vec("dog", wv_embeddings, tokenizer, dim=200)

array([ 0.68517721, -1.27789915, -0.41913974,  1.36231637, -3.16753983,
        0.09950767,  0.64026809, -1.12453389, -0.66996193, -0.69988519,
        0.49367711, -0.40500194, -3.07068157, -2.28099656,  0.85798043,
        2.70931077,  0.34927449, -0.03494101, -0.22330493,  1.2290467 ,
        1.77551568, -3.1583581 , -0.65159827,  0.72240961,  2.31930828,
       -1.79698622,  0.40903398, -2.74460411, -1.71799517, -0.91430902,
       -0.75887376, -0.35140672, -0.51827759, -1.90973508, -0.83007729,
        0.02147918,  1.17834711,  0.03169126, -0.30690229,  1.66662991,
        0.67113572, -2.17061329, -0.11800487,  0.22336982, -1.20753944,
       -0.86297905, -0.63865614,  1.17337942,  0.10022762,  0.70172793,
        2.72907281, -0.46404839, -2.17193055, -0.35628521, -1.84499574,
        0.10270727,  1.11255956, -0.8364318 ,  1.95134079, -0.97937447,
        1.26508594,  0.06809282,  0.64773178, -0.52431005, -0.61039591,
       -2.97982907, -0.78899652, -0.11004248,  1.76038408, -1.05

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [12]:
'''your code'''
round(question_to_vec("I love neural networks", wv_embeddings, tokenizer, dim=200)[2],2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Ответ: Чтобы получить максимум нужно, чтобы DCG@1 было минимум, то есть 0. Следовательно максимум = Hits@47 = 1.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$

In [13]:
print(round(1/np.log2(1+9),2))

0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$

In [14]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = sum([1 if rank <= k else 0 for rank in dup_ranks[:k]]) /  len(dup_ranks[:k])
    
    return hits_value

In [15]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = sum([1 / (np.log2(1+rank))  if rank <= k else 0 for rank in dup_ranks[:k]])  /  len(dup_ranks[:k])
#     dcg_value = np.sum([(2**rel - 1) / np.log2(rank + 2) for rank, rel in enumerate(dup_ranks)])

    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [16]:
import pandas as pd

In [17]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [candidates_ranking[0].index(answer) + 1 for answer in candidates_ranking[0] if answer == copy_answers[0]]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [18]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [19]:
# !unzip stackoverflow_similar_questions.zip

Считайте данные.

In [62]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        row = line.strip()
        row = row.split('\t')
        data.append(row)
    return data

Нам понадобиться только файл validation.

In [21]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [22]:
len(validation_data)

3760

Размер нескольких первых строк

In [23]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [25]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_vec = question_to_vec(question, embeddings, tokenizer, dim=200)
    result = []
    for q in candidates:
        q_vec = question_to_vec(q, embeddings, tokenizer, dim=200)
        similarity = cosine_similarity([q_vec,question_vec])
        result.append((similarity[0,1], q))
    result = sorted(result, reverse=True, key=lambda x: x[0])
    final_result = []
    for r in result:
        final_result.append((candidates.index(r[1]), r[1]))
    return final_result
        
    

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [26]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [27]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [28]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(1, 'Getting all list items of an unordered list in PHP'), #скрыт
            (0, 'select2 not displaying search results'), #скрыт
            (2, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Ответ: для `для эксперимента 2` 1,0,2

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [29]:
from tqdm.notebook import tqdm

In [30]:
validation_data[:5]

[['How to print a binary heap tree without recursion?',
  'How do you best convert a recursive function to an iterative one?',
  'How can i use ng-model with directive in angular js',
  'flash: drawing and erasing',
  'toggle react component using hide show classname',
  'Use a usercontrol from another project to current webpage',
  '~ Paths resolved differently after upgrading to ASP.NET 4',
  'Materialize datepicker - Rendering when an icon is clicked',
  'Creating PyPi package - Could not find a version that satisfies the requirement iso8601',
  'How can I analyze a confusion matrix?',
  'How do I declare a C array in Swift?',
  'Using rand() when flipping a coin and rolling a die',
  'Handling a JSON field with a special character in its name in Java',
  'React Native select row on ListView when push it',
  "Get 'creation_time' of video using ffmpeg and regex",
  'Does row exist and multiple where',
  "How to specify a classifier in a gradle dependency's dependency?",
  'Using $unw

In [31]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1) # на index 0 находится правильный элемент

  0%|          | 0/3760 [00:00<?, ?it/s]

In [32]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 0.326 | Hits@   5: 0.400
DCG@  10: 0.363 | Hits@  10: 0.400
DCG@ 100: 0.390 | Hits@ 100: 0.660
DCG@ 500: 0.412 | Hits@ 500: 0.870
DCG@1000: 0.434 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [33]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [34]:
train_data

[['converting string to list',
  'Convert Google results object (pure js) to Python object'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
  'Event handling for geometries in Three.js?'],
 ['Sending array via Ajax fails',
  'Getting all list items of an unordered list in PHP'],
 ['How to insert CookieCollection to CookieContainer?',
  'C# create cookie from string and send it'],
 ['Updating one element of a bound Observable collection',
  'WPF- How to update the changes in list item of a list'],
 ['MongoDB error on find()',
  'Retrieve only the queried element in an object array in MongoDB collection'],
 ['select2 not displaying search results',
  'How to use jQuery AJAX for an outside domain?'],
 ['Using Reduce to merge multiple data frames with passing arguments and without defining function outside the Reduce (syntax)',
  'R - merge a list of data frames into one data frame with missing values by row'],
 ['Adding Prototype to JavaScript Object L

In [35]:
'''your code'''
join_train_data = [tokenizer.tokenize(" ".join(questions)) for questions in train_data]

In [36]:
join_train_data

[['converting',
  'string',
  'to',
  'list',
  'Convert',
  'Google',
  'results',
  'object',
  '(',
  'pure',
  'js',
  ')',
  'to',
  'Python',
  'object'],
 ['Which',
  'HTML',
  '5',
  'Canvas',
  'Javascript',
  'to',
  'use',
  'for',
  'making',
  'an',
  'interactive',
  'drawing',
  'tool',
  '?',
  'Event',
  'handling',
  'for',
  'geometries',
  'in',
  'Three',
  '.',
  'js',
  '?'],
 ['Sending',
  'array',
  'via',
  'Ajax',
  'fails',
  'Getting',
  'all',
  'list',
  'items',
  'of',
  'an',
  'unordered',
  'list',
  'in',
  'PHP'],
 ['How',
  'to',
  'insert',
  'CookieCollection',
  'to',
  'CookieContainer',
  '?',
  'C',
  '#',
  'create',
  'cookie',
  'from',
  'string',
  'and',
  'send',
  'it'],
 ['Updating',
  'one',
  'element',
  'of',
  'a',
  'bound',
  'Observable',
  'collection',
  'WPF',
  '-',
  'How',
  'to',
  'update',
  'the',
  'changes',
  'in',
  'list',
  'item',
  'of',
  'a',
  'list'],
 ['MongoDB',
  'error',
  'on',
  'find',
  '()',
  

In [37]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(join_train_data, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [38]:
len(embeddings_trained.index_to_key)

52532

In [39]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [40]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 0.177 | Hits@   5: 0.400
DCG@  10: 0.353 | Hits@  10: 0.600
DCG@ 100: 0.416 | Hits@ 100: 0.690
DCG@ 500: 0.428 | Hits@ 500: 0.936
DCG@1000: 0.432 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


#### Какой принцип токенизации даёт качество лучше и почему?

In [41]:
from nltk.tokenize import WordPunctTokenizer
tk = WordPunctTokenizer() 
print(tk.tokenize("I can't believe it's Friday!"))

['I', 'can', "'", 't', 'believe', 'it', "'", 's', 'Friday', '!']


Этот токинайзер у нас в нашей работе

In [42]:
import re
print(re.findall('\w+', "I can't believe it's Friday!"))

['I', 'can', 't', 'believe', 'it', 's', 'Friday']


In [43]:
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

tokenizer = MyTokenizer()

train_data = read_corpus('./data/train.tsv')
join_train_data = [tokenizer.tokenize(" ".join(questions)) for questions in train_data]

embeddings_trained = Word2Vec(join_train_data, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [44]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 0.177 | Hits@   5: 0.400
DCG@  10: 0.267 | Hits@  10: 0.500
DCG@ 100: 0.417 | Hits@ 100: 0.700
DCG@ 500: 0.431 | Hits@ 500: 0.934
DCG@1000: 0.443 | Hits@1000: 1.000


In [45]:
from nltk.tokenize import word_tokenize
print(word_tokenize("I can't believe it's Friday!"))

['I', 'ca', "n't", 'believe', 'it', "'s", 'Friday', '!']


In [46]:
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return word_tokenize(text)

tokenizer = MyTokenizer()


train_data = read_corpus('./data/train.tsv')
join_train_data = [tokenizer.tokenize(" ".join(questions)) for questions in train_data]

embeddings_trained = Word2Vec(join_train_data, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [47]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 0.000 | Hits@   5: 0.000
DCG@  10: 0.305 | Hits@  10: 0.500
DCG@ 100: 0.393 | Hits@ 100: 0.680
DCG@ 500: 0.417 | Hits@ 500: 0.914
DCG@1000: 0.422 | Hits@1000: 1.000


re.findall() - лучше всего справились с задачей

Ответ: Тяжело сказать какой метод лучше. Каждый может повлиять по разному на другую задачу. Лучше конечно тот токенайзер сохраняет смысл целого слова. Но знаю, что другие модели даже работают по слогам.

#### Помогает ли нормализация слов?

In [48]:
import pymorphy2
import re
from gensim.models import Word2Vec


class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

tokenizer = MyTokenizer()
morph = pymorphy2.MorphAnalyzer()

def normalize_text(words): 
    normalized_words = [morph.parse(word)[0].normal_form for word in words]
    return normalized_words


train_data = read_corpus('./data/train.tsv')
join_train_data = [tokenizer.tokenize(" ".join(questions)) for questions in train_data]
normalized_data = [normalize_text(words) for words in join_train_data]



from gensim.models import Word2Vec
embeddings_trained = Word2Vec(normalized_data, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv



In [49]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [50]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 0.077 | Hits@   5: 0.200
DCG@  10: 0.206 | Hits@  10: 0.400
DCG@ 100: 0.322 | Hits@ 100: 0.600
DCG@ 500: 0.343 | Hits@ 500: 0.852
DCG@1000: 0.367 | Hits@1000: 1.000


Ответ: Да, конечно, это поможет эмбеддингам лучше и быстрее обучиться. Для больших данных это плюс. Но все-таки при нормализации теряется информация. У слов может быть много форм и возможно для наших данных этого не хватит, чтобы эмбеддинги передавали смыслы всех форм.

In [54]:
train_data = read_corpus('./data/train.tsv')

#### Какие эмбеддинги лучше справляются с задачей и почему?

In [51]:
pip install fasttext

Note: you may need to restart the kernel to use updated packages.


In [61]:
train_data = read_corpus('./data/train.tsv')

In [64]:
import re
import fasttext
import pymorphy2

# Define a function to read the corpus (assuming you have a function read_corpus)
def read_corpus(file_path):
    # Implement your corpus reading logic here
    pass

class MyTokenizer:
    def __init__(self):
        pass

    def tokenize(self, text):
        return re.findall(r'\w+', text)

tokenizer = MyTokenizer()
morph = pymorphy2.MorphAnalyzer()

def normalize_text(words):
    normalized_words = [morph.parse(word)[0].normal_form for word in words]
    return normalized_words

data = []
for line in open('./data/train.tsv', encoding='utf-8'):
    row = line.strip()
    row = row.split('\t')
    data.append(row)

train_data = data

join_train_data = [" ".join(questions) for questions in train_data]

normalized_data = [normalize_text(tokenizer.tokenize(text)) for text in join_train_data]

with open("normalized_data.txt", "w", encoding="utf-8") as file:
    for sentence in normalized_data:
        file.write(" ".join(sentence) + "\n")

model = fasttext.train_unsupervised(
    input="normalized_data.txt",  
    model='skipgram',
    dim=200,
    minCount=5,
    ws=5
)



wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, model, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)



Read 20M words
Number of words:  37335
Number of labels: 0
Progress: 100.0% words/sec/thread:   99829 lr:  0.000000 avg.loss:  1.781561 ETA:   0h 0m 0s% words/sec/thread:  100016 lr:  0.034272 avg.loss:  1.942295 ETA:   0h 1m18s


  0%|          | 0/3760 [00:00<?, ?it/s]

In [65]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 1.000 | Hits@   1: 1.000
DCG@   5: 0.200 | Hits@   5: 0.200
DCG@  10: 0.265 | Hits@  10: 0.400
DCG@ 100: 0.351 | Hits@ 100: 0.580
DCG@ 500: 0.408 | Hits@ 500: 0.862
DCG@1000: 0.430 | Hits@1000: 1.000


Ответ: FastText несильно помог.
На этой задаче у меня получилась одинаковая метрика. Это и логично. SO эмбеддинги обучаны на вопросах из SO и наш Word2vec обучен на вопросах SO из нашего файла. 

#### Почему получилось плохое качество решения задачи?

Ответ: Думаю недобучение. Метрики одинаковые при разных эмбеддингах и токенайзерах. Скорее заучил очень популярные слова.

#### Предложите свой подход к решению задачи.

Ответ: Использовать не Word2vec, какие-то более сложные модели. Почистить лучше текст. 